<a href="https://colab.research.google.com/github/JeisonTantachuco/CursoGEE/blob/main/module06/01_composites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: Composicion y mosaicos**

En esta lectura, aprenderemos sobre:

- Como realizar mosaicos espaciales.
- Como realizar un mosaico considerando imagenes Landsat 8 y Landsat 7.

<img src="https://user-images.githubusercontent.com/16768318/73668658-120f6f80-469e-11ea-9c31-84ef520817f2.jpg" align="right" width = 60%/>

In [1]:
#@title Credenciales Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize(project = 'ee-jtantaroman')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

En general, la **composicion** se refiere al proceso de combinar imagenes superpuestas espacialmente en una sola imagen basada en una funcion de agregacion. El **mosaico** se refiere al proceso de ensamblar conjuntos de datos de imagenes **espacialmente** para producir una imagen espacialmente continua. En Earth Engine, estos terminos se usan indistintamente, aunque se admiten tanto la composicion como el mosaico. Por ejemplo, considere la tarea de **componer varias imagenes en la misma ubicacion**. El siguiente ejemplo demuestra como hacer un compuesto cuya funcion de agregacion es tomar el maximo valor:

In [3]:
# Cargue los datos NAIP
naip2004_2012 = ee.ImageCollection('USDA/NAIP/DOQQ')\
                  .filterBounds(ee.Geometry.Point(-71.08841, 42.39823))\
                  .filterDate('2004-07-01', '2012-12-31')\
                  .select(['R', 'G', 'B'])

# Realize un "composite" temporal de las imagenes con una funcion de valor maximo.
composite = naip2004_2012.max()
center = [-71.12532, 42.3712]
mapdisplay(center, {'max value composite':composite.getMapId()},zoom_start=12)

Considere la necesidad de mosaico para cuatro diferentes escenas realizadas al mismo tiempo, pero en diferentes ubicaciones. El siguiente ejemplo demuestra que usando el metodo `imageCollection.mosaic()` puede solucionarlo:

In [4]:
#El poligono en azul es nuestro ambito de estudio y vemos que es necesario mas de una imagen para poder traslapar completamente nuestro ambito
roi = ee.Geometry.Rectangle(-71.17965, 42.35125, -71.08824, 42.40584)
mapdisplay(center,{'roi':roi.getInfo(), 'composite_01':composite.getMapId()}, zoom_start=12)

In [5]:
# Cargue los datos NAIP, aca al usar filter bounds estamos cargando todas las imagenes que traslapen con nuestro ambito
naip2012 = ee.ImageCollection('USDA/NAIP/DOQQ')\
             .filterBounds(roi)\
             .filterDate('2012-01-01', '2012-12-31')

# Creamos un mosaico espacial y displayamos, con esto unimos (mosaico) todas las imagenes
mosaic = naip2012.mosaic() #si no performamos el mosaico podemos ver el mismo resultado, visualmente se ve igual, pero en realidad al hacer mosaico y descargar nos descargara solo 1 imagen, a diferencia de antes del mosaico que nos descargaria varias imagenes
center = [-71.12532, 42.3712]
mapdisplay(center,{'roi':roi.getInfo(), 'spatial mosaic':mosaic.getMapId()},zoom_start=12)

### **Ejercicio:**
**Crear un composite del año 2018, en la selva de Peru, considerando los datos de las imagenes Landsat. Automatizar el proceso mediante una funcion.**

In [26]:
#  Parametros globales
range_date = ['2018-01-01', '2018-12-31']
center = [-72.628,-10.609]
roi = ee.Geometry.Point(center)
# Parametros de visualizacion de landat7 landsat8 y landast7 y 8
l7_viz_params = {'bands':['SR_B5','SR_B4','SR_B3'],'min':0,'max': 0.4, 'gamma':1.2}
l8_viz_params = {'bands':['B6','B5','B4'],'min':0,'max': 4000, 'gamma':1.2}
l78_viz_params = {'bands':['SWIR','NIR','RED'],'min':0,'max': 4000, 'gamma':1.2}

Funcion para obtener el tiempo de cada imagen

In [27]:
from datetime import datetime as dt
def get_dates(ic):
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))

In [28]:
## Ahora ha cambiado L7 de C01 a C02 y se necesita aplicar los factores de escala en la cual se guardan los datos
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T1_L2#image-properties
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

  return image.addBands(opticalBands, None, True)\
              .addBands(thermalBand, None, True)

##### **Landsat 7 - sin mascara**

Vemos que hay muchas nubes en la imagen y sombras es por eso que sera necesario aplicar una mascara

In [38]:
l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
       .filterBounds(roi)\
       .filterDate(range_date[0], range_date[1])\
       .filterMetadata('CLOUD_COVER','less_than',20)\
       .map(applyScaleFactors)

#Además estamos viendo que hay pocas imagenes para la selva peruana, solo 4 imagenes cumplen con la covertura de nube menor al 20%
get_dates(l7)

['2018-07-26 14:57:07',
 '2018-08-11 14:56:49',
 '2018-08-27 14:56:30',
 '2018-09-12 14:56:10']

In [30]:
# Displaye los resultados
to_plot_median = l7.median().getMapId(l7_viz_params)
mapdisplay(center,{'median_l7_cloud':to_plot_median})

##### **Landsat 7 - con mascara**

Al hacer la mascara se quian los pixeles donde encuentran todo tipo de nubes que se han enumerado en la funcion, es por eso que nos queda algunos huecos en las imagenes

In [35]:
def l7_maskcloud(image):
  qa = image.select('QA_PIXEL')
  #Generando una mascara para solo aquellos pixeles que no contienen cloud shadow, cloud y cloud confidence
  #Ahora cambio y el primero es cloud (bit3) el segundo cloud shadow (bit4) , cloud confidence (bit9), cloud shadow confidence (bit11) y cirrus confidence (bit15)
  my_mask = (1 << 3) + (1 << 4) + (1 << 9) + (1 << 11) + (1 << 15)
  mask = qa.bitwiseAnd(my_mask).eq(0)
  ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
  return image.select(["SR_B.*"]).addBands(ndvi).updateMask(mask).copyProperties(image, ["system:time_start"])

In [36]:
l7_nocloud = l7.map(l7_maskcloud)
#El qualityMosaic ordena las imagenes al hacer una mosaico en un image collection
l7_nocloud_nvdi = l7_nocloud.qualityMosaic('NDVI')
l7_nocloud_getmapid = l7_nocloud_nvdi.getMapId(l7_viz_params)
mapdisplay(center,{'ndvi_l7_nocloud':l7_nocloud_getmapid,
                   'median_l7_cloud':to_plot_median})

##### **Landsat 8 - sin mascara**

In [ ]:
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
       .filterBounds(roi)\
       .filterDate(range_date[0], range_date[1])\
       .filterMetadata('CLOUD_COVER','less_than',20)

In [ ]:
# Displaye los resultados
l8_to_plot_median = l8.median().getMapId(l8_viz_params)
mapdisplay(center,{'median_l8_cloud':l8_to_plot_median})

##### **Landsat 8 - con mascara**

In [ ]:
def l8_maskcloud(image):
  qa = image.select('pixel_qa')
  my_mask = (1 << 3) + (1 << 5) + (1 << 7) + (1 << 9) + (1 << 10)
  mask = qa.bitwiseAnd(my_mask).eq(0)
  ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
  return image.select(["B.*"]).addBands(ndvi).updateMask(mask).copyProperties(image, ["system:time_start"])

In [ ]:
l8_nocloud = l8.map(l8_maskcloud)
l8_nocloud_nvdi = l8_nocloud.qualityMosaic('NDVI')
l8_nocloud_getmapid = l8_nocloud_nvdi.getMapId(l8_viz_params)
mapdisplay(center,{'ndvi_l8_nocloud':l8_nocloud_getmapid,
                   'median_l8_cloud':l8_to_plot_median})

##### **Landsat 7 y 8 - con mascara**

In [ ]:
def select_image_l8(img):
  return img.select(['B6','B5','B4','NDVI']).rename('SWIR','NIR','RED','NDVI')
def select_image_l7(img):
  return img.select(['B5','B4','B3','NDVI']).rename('SWIR','NIR','RED','NDVI')

l7_veg = l7_nocloud.map(select_image_l7)
l8_veg = l8_nocloud.map(select_image_l8)
l78 =l8_veg.merge(l7_veg)

In [ ]:
l78_nocloud_nvdi = l78.qualityMosaic('NDVI')
l78_nocloud_median = l78.median()
l78_nocloud_getmapid_ndvi = l78_nocloud_nvdi.getMapId(l78_viz_params)
l78_nocloud_getmapid_median = l78_nocloud_median.getMapId(l78_viz_params)
mapdisplay(center,{'l78_nocloud_getmapid_ndvi':l78_nocloud_getmapid_ndvi,
                   'l78_nocloud_getmapid_median':l78_nocloud_getmapid_median})

### **Funcion para generar composites a partir de L7 y L8 - vegetacion**

In [ ]:
def composite_l7l8_vegetation(init_date, last_date, roi, cloud_per, calendar, composite = 'median'):
  '''
  Crear un composite de vegetacion a partir de datos Landsat 8 y Landsat  7

  init_date: Fecha de inicio
  last_date: Fecha de fin
  roi: Ambito de estudio
  cloud_per: Porcentaje de nubes
  calendar: Lista numerica de dos elementos. Que indica los meses a filtrar.
  composite: Funcion utilizada para realizar el composite. Actualmente
             disponible: mean, ndvi (max) y none (no composite).
  '''
  def l7_maskcloud(image):
    qa = image.select('pixel_qa')
    my_mask = (1 << 3) + (1 << 5) + (1 << 7)
    mask = qa.bitwiseAnd(my_mask).eq(0)
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)\
                .select(['B5','B4','B3','NDVI'])\
                .rename('SWIR','NIR','RED','NDVI')\
                .updateMask(mask)\
                .copyProperties(image, ["system:time_start"])
  def l8_maskcloud(image):
    qa = image.select('pixel_qa')
    my_mask = (1 << 3) + (1 << 5) + (1 << 7) + (1 << 9) + (1 << 10)
    mask = qa.bitwiseAnd(my_mask).eq(0)
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)\
                .select(['B6','B5','B4','NDVI'])\
                .rename('SWIR','NIR','RED','NDVI')\
                .updateMask(mask)\
                .copyProperties(image, ["system:time_start"])
  l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
        .filterBounds(roi)\
        .filterDate(init_date, last_date)\
        .filterMetadata('CLOUD_COVER','less_than',cloud_per)\
        .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
        .map(l7_maskcloud)
  l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterBounds(roi)\
        .filterDate(init_date, last_date)\
        .filterMetadata('CLOUD_COVER','less_than',cloud_per)\
        .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
        .map(l8_maskcloud)
  l78 =l8.merge(l7)
  if composite == 'median':
    return l78.median()
  elif composite == 'ndvi':
    return l78.qualityMosaic('NDVI')
  else:
    return l78

Correr **composite_l7l8_vegetation** para una area determinada

In [ ]:
# ------------------------------
#  PARAMETROS GLOBALES
# ------------------------------
range_date = ['2019-01-01', '2019-12-31'] #fecha de inicio y fin
center = [-72.628,-10.609]
roi = ee.Geometry.Point(center) #Area de interes (seleccione el path and row)
calendar = [4,11] # Seleccionar imagenes de Abril a Noviembre
cloud_per = 20 # Maximo porcentaje de cobertura de nubes a considerar
composite = 'median' # Funcion para generar el composite (median, ndvi)

In [ ]:
composite = composite_l7l8_vegetation(init_date=range_date[0],
                                      last_date=range_date[1],
                                      roi=roi,
                                      cloud_per=cloud_per,
                                      calendar=calendar,
                                      composite=composite)

In [ ]:
l78_viz_params = {'bands':['SWIR','NIR','RED'],'min':0,'max': 4000, 'gamma':1.2}
mapdisplay(center,{'composite_median':composite.getMapId(l78_viz_params)})

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>